<div class="markdown-google-sans">

## PCP: Prediction of Cardiac Problems
</div>

aiUTEChallenge ML – Aprendizaje Supervisado
    

<div class="markdown-google-sans">

## Objetivo
</div>


Se deberá completar la columna EnfermedadCorazon del archivo df_data.CSV para cada registro categorizando en si es probable o no que desarrolle enfermedad al igual que como se tiene en el dataset de entrenamiento.

* Explorar e identificar las características relevantes para predecir la columna EnfermedadCorazon

* Comprender los datos:

  1. **Edad**: edad del paciente ✴ `[años]`
  2. **Sexo**: sexo del paciente ✴ `[M: Masculino, F: Femenino]`
  3. **TipoDolorPecho**: tipo de dolor de pecho ✴ `[TA: Angina Típica, ATA: Angina Atípica, NAP: Dolor No Anginoso, ASY: Asintomático]`
  4. **PresionRep**: presión arterial en reposo ✴ `[mm Hg]`
  5. **Colesterol**: colesterol sérico ✴ `[mm/dl]`
  6. **GlucosaEnAyunas**: glucosa en sangre en ayunas ✴ `[1: si GlucosaEnAyunas > 120 mg/dl, 0: de lo contrario]`
  7. **ECGRep**: resultados del electrocardiograma en reposo ✴ `[Normal: Normal, ST: con anormalidad en la onda ST-T (inversiones de la onda T y/o elevación o depresión del ST > 0.05 mV), LVH: mostrando probable o definitiva hipertrofia ventricular izquierda según los criterios de Estes]`
  8. **FCMaxima**: frecuencia cardíaca máxima alcanzada ✴ `[Valor numérico entre 60 y 202]`
  9. **AnginaEjercicio**: angina inducida por ejercicio ✴ `[Y: Sí, N: No]`
  10. **DescensoST**: depresión antigua = ST ✴ `[Valor numérico medido en depresión]`
  11. **PendienteST**: la pendiente del segmento ST durante el ejercicio máximo ✴ `[Up: Ascendente, Flat: Plano, Down: Descendente]`
  12. **EnfermedadCardiaca**: clase de salida ✴ `[1: enfermedad cardíaca, 0: Normal]`

  Ref. **punto 7**: Criterios de Romhilt-Estes para la hipertrofia del ventrículo izquierdo




<div class="markdown-google-sans">

## Contenido
</div>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol>
    <li><a href="#Importación_datos">Importación de los datos</a></li>
    <li><a href="#Exploración_procesamiento">Exploración y procesamiento de los datos</a></li>
    <li><a href="#Análisis_patrones">Análisis de patrones de características individuales mediante visualización</a></li>
    <li><a href="#Desarrollo_modelos">Desarrollo de Modelos</a></li>
    <li><a href="#Evaluación_modelo">Evaluación del Modelo y predicciones</a></li>
</ol>

</div>

<hr>


<div class="markdown-google-sans">

## Importación de los datos
</div>


Librerías:


In [ ]:
#install specific version of libraries used in lab
#! mamba install pandas==1.3.3
#! mamba install numpy=1.21.2
#! mamba install scipy=1.7.1-y
#! mamba install seaborn=0.9.0-y
#! mamba install seaborn=0.9.0-y

In [ ]:
import pandas as pd
import numpy as npz

Cargar y explorar el conjunto de datos "df_data.csv". Dataset: `df`:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# seleccionar la ruta del dataset "df_data.csv"
path='.../df_data.csv'
df = pd.read_csv(path)
df.head()

<div class="markdown-google-sans">

## 2. Exploración y procesamiento de los datos:
</div>

* Procesamiento de los Datos
* Manejo de valores faltantes
* Manejo de valores atípicos
* Codificación de variables categóricas
* Identificación de las características relevantes para la solución del problema


In [ ]:
# ver el conjunto de datos
df.head(10)

In [ ]:
# identificar valores faltantes
df.isnull().values.any()

In [ ]:
print(df.isnull().sum())

In [ ]:
missing_data = df.isnull()
missing_data.head(5)

In [ ]:
# contar valores faltantes
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

In [ ]:
# valores únicos para cada columna
df.nunique()

In [ ]:
# información general del dataset
print(' - Cantidad de Filas y columnas:', df.shape, '\n')
print(' - Nombre de las columnas:',df.columns, '\n')
print(' - Información sobre el dataset:', '\n')
df.info()

In [ ]:
# búsqueda de entradas de datos (object)
for x in df.columns:
    if df[x].dtype == 'object':
        print("Nombre de columna:", x)
        print(df[x].value_counts())
        print("-------------------------")

In [ ]:
# buscar duplicados
df.duplicated().value_counts

Según las *exploraciones* anteriores el dataframe no tiene datos faltantes ni datos duplicados

In [ ]:
# tipos de datos por columna
print(df.dtypes)
df.head(5)

In [ ]:
# contar tipos de datos
numericos = df.select_dtypes(include=['int', 'float']).shape[1]
objetos = df.select_dtypes(include='object').shape[1]
print(f"Número de columnas numéricas: {numericos}")
print(f"Número de columnas de tipo objeto: {objetos}")

In [ ]:
print("\nResumen estadístico:")
print(df.describe())

In [ ]:
print("\nResumen estadístico categóricas:")
print(df.describe(include=['object']))

In [ ]:
# variable indicadora (One-Hot Encoding), conversión de variable Sexo en formato numérico:
dummy_variable_1 = pd.get_dummies(df["Sexo"])
dummy_variable_1.head()

In [ ]:
# cambiar nombre de las columnas
dummy_variable_1.rename(columns={'F':'Femenino', 'M':'Masculino'}, inplace=True)
dummy_variable_1.head()

In [ ]:
# unir data frame "df" y "dummy_variable_1"
df = pd.concat([df, dummy_variable_1], axis=1)

# eliminar columna original "Sexo" del dataframe
df.drop('Sexo', axis = 1, inplace=True)
df.head()

In [ ]:
# conversión de variable AnginaEjercicio en formato numérico:
dummy_variable_2 = pd.get_dummies(df["AnginaEjercicio"])
dummy_variable_2.head()

In [ ]:
# cambiar nombre de las columnas
dummy_variable_2.rename(columns={'N':'AnginaEjercicio_No', 'Y':'AnginaEjercicio_Si'}, inplace=True)
dummy_variable_2.head()

In [ ]:
# unir data frame "df" y "dummy_variable_2"
df = pd.concat([df, dummy_variable_2], axis=1)

# eliminar columna original "AnginaEjercicio" del dataframe
df.drop('AnginaEjercicio', axis = 1, inplace=True)
df.head()

In [ ]:
# normalización: Escalar las columnas PresionRep, Colesterol, FCMaxima y DescensoST
df['PresionRep'] = df['PresionRep']/df['PresionRep'].max()
df['Colesterol'] = df['Colesterol']/df['Colesterol'].max()
df['FCMaxima'] = df['FCMaxima']/df['FCMaxima'].max()
df['DescensoST'] = df['DescensoST']/df['DescensoST'].max()
df.head(10)

In [ ]:
# tipos de datos por columna
print(df.dtypes)
df.head(5)

In [ ]:
# conversión a de variables uint8 a int64
df[["Femenino", "Masculino", "AnginaEjercicio_No", "AnginaEjercicio_Si"]] = df[["Femenino", "Masculino", "AnginaEjercicio_No", "AnginaEjercicio_Si"]].astype("int")
print(df.dtypes)

<div class="markdown-google-sans">

## 3. Análisis de patrones de características individuales mediante visualización
</div>

In [ ]:
# importación de librerías para graficar
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
# gráfico de distribución para cada variable numérica

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(12, 7))
axes = axes.flat
columnas_numeric = df.select_dtypes(include=['float64', 'int64']).columns
columnas_numeric = columnas_numeric.drop('EnfermedadCorazon')

for i, colum in enumerate(columnas_numeric):
    sns.histplot(
        data     = df,
        x        = colum,
        stat     = "count",
        kde      = True,
        color    = (list(plt.rcParams['axes.prop_cycle'])*2)[i]["color"],
        line_kws = {'linewidth': 2},
        alpha    = 0.3,
        ax       = axes[i]
    )
    axes[i].set_title(colum, fontsize = 7, fontweight = "bold")
    axes[i].tick_params(labelsize = 6)
    axes[i].set_xlabel("")

# Se eliminan los axes vacíos
for i in [10, 11]:
  fig.delaxes(axes[i])


fig.tight_layout()
plt.subplots_adjust(top = 0.9)
fig.suptitle('Distribución variables numéricas', fontsize = 10, fontweight = "bold");

In [ ]:
# gráfico para cada variable cualitativa

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
axes = axes.flat
columnas_object = df.select_dtypes(include=['object']).columns

for i, colum in enumerate(columnas_object):
    df[colum].value_counts().plot.barh(ax = axes[i])
    axes[i].set_title(colum, fontsize = 7, fontweight = "bold")
    axes[i].tick_params(labelsize = 6)
    axes[i].set_xlabel("")

fig.tight_layout()
plt.subplots_adjust(top=0.9)
fig.suptitle('Distribución variables cualitativas',fontsize = 10, fontweight = "bold");

In [ ]:
# tabla de frecuencia relativa de variable TipoDolorPecho
100 * df["TipoDolorPecho"].value_counts() / len(df["TipoDolorPecho"])

In [ ]:
# tabla de contingencia TipoDolorPecho / EnfermedadCorazon
pd.crosstab(index=df['EnfermedadCorazon'],
            columns=df['TipoDolorPecho'], margins=True)

In [ ]:
# tabla de contingencia en porcentajes relativos total
pd.crosstab(index=df['EnfermedadCorazon'],
            columns=df['TipoDolorPecho'],
            margins=True).apply(lambda r: r/len(df) *100, axis=1)

In [ ]:
# tabla de frecuencia relativa de variable ECGRep
100 * df["ECGRep"].value_counts() / len(df["ECGRep"])

In [ ]:
# tabla de contingencia ECGRep / EnfermedadCorazon
pd.crosstab(index=df['EnfermedadCorazon'],
            columns=df['ECGRep'], margins=True)

In [ ]:
# tabla de contingencia en porcentajes relativos total
pd.crosstab(index=df['EnfermedadCorazon'],
            columns=df['ECGRep'],
            margins=True).apply(lambda r: r/len(df) *100, axis=1)

In [ ]:
# tabla de frecuencia relativa de variable ECGRep
100 * df["PendienteST"].value_counts() / len(df["PendienteST"])

In [ ]:
# tabla de contingencia PendienteST / EnfermedadCorazon
pd.crosstab(index=df['EnfermedadCorazon'],
            columns=df['PendienteST'], margins=True)

In [ ]:
# tabla de contingencia en porcentajes relativos total
pd.crosstab(index=df['EnfermedadCorazon'],
            columns=df['PendienteST'],
            margins=True).apply(lambda r: r/len(df) *100, axis=1)

<p>Gráfico de dispersión para visualizar la consentración de cada una de las características seleccionadas con la variable objetivo.</p>


In [ ]:
# gráfico variables numéricas y variable objetivo
variables = ["Edad", "PresionRep","Colesterol", "FCMaxima","EnfermedadCorazon", "DescensoST"]
sns.pairplot(df[variables], height=3, diag_kind= "kde", hue="EnfermedadCorazon")

In [ ]:
# tabla de frecuencia relativa de variable EnfermedadCorazon
100 * df["EnfermedadCorazon"].value_counts() / len(df["EnfermedadCorazon"])

In [ ]:
# codificar las variables categoricas
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["TipoDolorPecho_CAT"] = le.fit_transform(df["TipoDolorPecho"])
df["ECGRep_CAT"] = le.fit_transform(df["ECGRep"])
df["PendienteST_CAT"] = le.fit_transform(df["PendienteST"])
df.head()

In [ ]:
# eliminar las columnas originales del dataframe
df.drop('TipoDolorPecho', axis = 1, inplace=True)
df.drop('ECGRep', axis = 1, inplace=True)
df.drop('PendienteST', axis = 1, inplace=True)
df.head()

Correlación entre las variables de tipo
"int64" o "float64":


In [ ]:
# visualizar la correlación entre las variables de tipo "int64" o "float64":
df.corr()

In [ ]:
# matriz de correlación:
corrmat = df.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True, annot=True);

In [ ]:
# correlaciones por orden
corr = df.corr()
corr[['EnfermedadCorazon']].sort_values(by = 'EnfermedadCorazon',ascending = False)\
.style.background_gradient()

In [ ]:
# importacion libreria scipy
from scipy import stats

In [ ]:
# coeficiente Pearson - valor de P
pearson_coef, p_value = stats.pearsonr(df['Edad'], df['EnfermedadCorazon'])
print("El coeficiente de Pearson es", pearson_coef, " con un valor de P =", p_value)

In [ ]:
pearson_coef, p_value = stats.pearsonr(df['AnginaEjercicio_Si'], df['EnfermedadCorazon'])
print("El coeficiente de Pearson es", pearson_coef, " con un valor de P =", p_value)

In [ ]:
pearson_coef, p_value = stats.pearsonr(df['FCMaxima'], df['EnfermedadCorazon'])
print("El coeficiente de Pearson es", pearson_coef, " con un valor de P =", p_value)

In [ ]:
pearson_coef, p_value = stats.pearsonr(df['PendienteST_CAT'], df['EnfermedadCorazon'])
print("El coeficiente de Pearson es", pearson_coef, " con un valor de P =", p_value)

In [ ]:
# evaluamos a través de un gráfico si la edad aumenta la probabilidad de enfermedad al corazón
sns.histplot(data=df[df['EnfermedadCorazon'] == 0], x='Edad', kde=True, color='blue', label='Sin enfermedad')
sns.histplot(data=df[df['EnfermedadCorazon'] == 1], x='Edad', kde=True, color='red', label='Con enfermedad')
plt.title('Edad como probabilidad de enfermedad al corazón')
plt.xlabel('Edad')
plt.ylabel('Cantidad(Frecuencia)')
plt.legend()
plt.show()

In [ ]:
# evaluamos a través de un gráfico si la variable colesterol aumenta la probabilidad de enfermedad al corazón
sns.histplot(data=df[df['EnfermedadCorazon'] == 0], x='Colesterol', kde=True, color='blue', label='Sin enfermedad')
sns.histplot(data=df[df['EnfermedadCorazon'] == 1], x='Colesterol', kde=True, color='red', label='Con enfermedad')
plt.title('Colesterol como probabilidad de enfermedad al corazón')
plt.xlabel('Colesterol')
plt.ylabel('Cantidad(Frecuencia)')
plt.legend()
plt.show()

In [ ]:
# analizamos las variables Edad y GlucosaEnAyunas, Sin efermedad o Con enfermedad
# import plotly.express as px
fig =px.violin(df, y="Edad", x="GlucosaEnAyunas",color="EnfermedadCorazon",box= True, points="all",hover_data= df.columns)
fig.update_layout(title_text = "Analisis Edad y GlucosaEnAyunas - Sin efermedad o Con enfermedad")
fig.show()

In [ ]:
# import plotly.graph_objs as go

# personas que tienen o no tienen GlucosaEnAyunas
con_GlucosaEnAyunas = df[df['GlucosaEnAyunas']==1]
sin_GlucosaEnAyunas = df[df['GlucosaEnAyunas']==0]

labels = ['Con GlucosaEnAyunas','Sin GlucosaEnAyunas']
values = [len(con_GlucosaEnAyunas), len(sin_GlucosaEnAyunas)]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(
    title_text="Con GlucosaEnAyunas - Sin GlucosaEnAyunas",
    autosize=False,
    width=600,
    height=300,
)

fig.show()

# eventos de enfermedad al corazón por GlucosaEnAyunas
fig = px.pie(df, values='GlucosaEnAyunas',names='EnfermedadCorazon', title='Eventos de enfermedad al corazón por GlucosaEnAyunas')
fig.update_layout(
    autosize=False,
    width=600,
    height=300,
)

fig.show()

In [ ]:
# visualizando nuevemente el df
df.head(25)

<div class="markdown-google-sans">

## 4. Desarrollo de Modelos
</div>

Probando Regresión Lineal

In [ ]:
# definimos x , y
y_data = df['EnfermedadCorazon']

In [ ]:
x_data=df.drop('EnfermedadCorazon',axis=1)

In [ ]:
# dividimos  el conjunto de datos de modo que el 20% de las muestras de datos se utilicen para las pruebas
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.20, random_state=1)

print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])

In [ ]:
# importar LinearRegression
from sklearn.linear_model import LinearRegression

In [ ]:
# creamos el objeto de regresión lineal
lre=LinearRegression()

In [ ]:
# ajustamos el modelo utilizando AnginaEjercicio_Si
lre.fit(x_train[['AnginaEjercicio_Si']], y_train)

In [ ]:
# calculamos el R^2 en los datos de prueba
lre.score(x_test[['AnginaEjercicio_Si']], y_test)

In [ ]:
# el R ^ 2 es más pequeño usando los datos de prueba en comparación con los datos de entrenamiento.
lre.score(x_train[['AnginaEjercicio_Si']], y_train)

In [ ]:
# probamos el R^2 en los datos de prueba utilizando el 30% del conjunto de datos para realizar pruebas.
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_data, y_data, test_size=0.30, random_state=0)
lre.fit(x_train1[['AnginaEjercicio_Si']],y_train1)
lre.score(x_test1[['AnginaEjercicio_Si']],y_test1)

Probando Regresión Logística

In [ ]:
# creamos otro dataframe para probar
df1=df

In [ ]:
# definimos x , y
y_data1 = df1['EnfermedadCorazon']

In [ ]:
x_data1=df1.drop('EnfermedadCorazon',axis=1)

In [ ]:
# dividimos el conjunto de datos de manera que el 20% de las muestras de datos se utilicen para las pruebas
from sklearn.model_selection import train_test_split

x_train2, x_test2, y_train2, y_test2 = train_test_split(x_data1, y_data1, test_size=0.20, random_state=1)

print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])

In [ ]:
# importar LogisticRegression
from sklearn.linear_model import LogisticRegression

In [ ]:
# creamos el objeto de regresión
mlreg =LogisticRegression()

In [ ]:
# ajustamos el modelo
mlreg.fit(x_train2, y_train2)

In [ ]:
# predicciones
y_test_pred = mlreg.predict(x_test2)

In [ ]:
# exactitud del modelo
occuracy = mlreg.score(x_test2, y_test2)
print("Exactitud: ", occuracy)

In [ ]:
# precisión del modelo
from sklearn.metrics import precision_score
pre =precision_score(y_test2, y_test_pred)
print("Precisión: ", pre)

In [ ]:
# sensibilidad del modelo
from sklearn.metrics import recall_score
sens = recall_score(y_test2, y_test_pred)
print("Sensibilidad: ", sens)

In [ ]:
# puntaje del modelo
from sklearn.metrics import f1_score
puntaje = f1_score(y_test2, y_test_pred)
print("Puntaje: ", puntaje)

In [ ]:
# evaluación del modelo con matriz de confusión
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test2, y_test_pred))

Valores obtenidos en la matríz es de: 252 predichos correctos y 42 incorrectos

Probando el algoritmo SVM

In [ ]:
# creamos un df con todas las variables
df2=df

In [ ]:
# definimos x , y
y_data3 = df2['EnfermedadCorazon']

In [ ]:
x_data3=df2.drop('EnfermedadCorazon',axis=1)

In [ ]:
# dividimos el conjunto de datos de manera que el 20% de las muestras de datos se utilicen para las pruebas
from sklearn.model_selection import train_test_split

x_train3, x_test3, y_train3, y_test3 = train_test_split(x_data3, y_data3, test_size=0.20, random_state=1)

print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])

In [ ]:
# importar SVM
from sklearn import svm

In [ ]:
# creamos el objeto de regresión
algoritmo = svm.SVC(kernel="linear")

In [ ]:
# entrenamos el algoritmo
algoritmo.fit(x_train3, y_train3)

In [ ]:
# predicciones
y_test_pred3 = algoritmo.predict(x_test3)

In [ ]:
# exactitud del modelo
occuracy_svc = mlreg.score(x_test3, y_test3)
print("Exactitud: ", occuracy_svc)

In [ ]:
# precisión del modelo
from sklearn.metrics import precision_score
pre_svc =precision_score(y_test3, y_test_pred3)
print("Precisión: ", pre_svc)

In [ ]:
# sensibilidad del modelo
from sklearn.metrics import recall_score
sens_svc = recall_score(y_test3, y_test_pred3)
print("Sensibilidad: ", sens_svc)

In [ ]:
# puntaje del modelo
from sklearn.metrics import f1_score
puntaje_svc = f1_score(y_test3, y_test_pred3)
print("Puntaje: ", puntaje_svc)

In [ ]:
# evaluación del modelo con matriz de confusión
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test3, y_test_pred3))

Valores obtenidos en la matríz es de: 260 predichos correctos y 34 incorrectos

<div class="markdown-google-sans">

## 5. Evaluación del Modelo y predicciones
</div>

In [ ]:
# calculamos el MSE par evaluar el modelo
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test2, y_test_pred)
print("Error Cuadrático Medio (MSE):", mse)

In [ ]:
# importamos el dataset de evaluación
p='/content/drive/MyDrive/Colab Notebooks/st_verify_data.csv'
st_verify_data = pd.read_csv(p)
st_verify_data.head()

In [ ]:
# verificamos dataset
print(' - Cantidad de Filas y columnas:', st_verify_data.shape, '\n')

In [ ]:
# creamos el df_st_verify_data (dataset de evaluación) con las mismas variables
df_st_verify_data = st_verify_data
df_st_verify_data.head()

In [ ]:
# verificamos dataset
print(' - Cantidad de Filas y columnas:', df_st_verify_data.shape, '\n')

Procesamos las columnas del dataframe "df_st_verify_data" igual que con con las columnas de dataframe "df_data.csv"

In [ ]:
# variable indicadora (One-Hot Encoding), conversión de variable Sexo en formato numérico:
dummy_variable_3 = pd.get_dummies(df_st_verify_data["Sexo"])
dummy_variable_3.head()

In [ ]:
# cambiar nombre de las columnas
dummy_variable_3.rename(columns={'F':'Femenino', 'M':'Masculino'}, inplace=True)
dummy_variable_3.head()

In [ ]:
# unir data frame "df_st_verify_data" y "dummy_variable_3"
df_st_verify_data = pd.concat([df_st_verify_data, dummy_variable_3], axis=1)

# eliminar columna original "Sexo" del dataframe
df_st_verify_data.drop('Sexo', axis = 1, inplace=True)
df_st_verify_data.head()

In [ ]:
# conversión de variable AnginaEjercicio en formato numérico:
dummy_variable_4 = pd.get_dummies(df_st_verify_data["AnginaEjercicio"])
dummy_variable_4.head()

In [ ]:
# cambiar nombre de las columnas
dummy_variable_4.rename(columns={'N':'AnginaEjercicio_No', 'Y':'AnginaEjercicio_Si'}, inplace=True)
dummy_variable_4.head()

In [ ]:
# unir data frame "df_st_verify_data" y "dummy_variable_4"
df_st_verify_data = pd.concat([df_st_verify_data, dummy_variable_4], axis=1)

# eliminar columna original "AnginaEjercicio" del dataframe
df_st_verify_data.drop('AnginaEjercicio', axis = 1, inplace=True)
df_st_verify_data.head()

In [ ]:
# normalización: Escalar las columnas PresionRep, Colesterol, FCMaxima y DescensoST
df_st_verify_data['PresionRep'] = df_st_verify_data['PresionRep']/df_st_verify_data['PresionRep'].max()
df_st_verify_data['Colesterol'] = df_st_verify_data['Colesterol']/df_st_verify_data['Colesterol'].max()
df_st_verify_data['FCMaxima'] = df_st_verify_data['FCMaxima']/df_st_verify_data['FCMaxima'].max()
df_st_verify_data['DescensoST'] = df_st_verify_data['DescensoST']/df_st_verify_data['DescensoST'].max()
df_st_verify_data.head(10)

In [ ]:
# codificar las variables categoricas
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_st_verify_data["TipoDolorPecho_CAT"] = le.fit_transform(df_st_verify_data["TipoDolorPecho"])
df_st_verify_data["ECGRep_CAT"] = le.fit_transform(df_st_verify_data["ECGRep"])
df_st_verify_data["PendienteST_CAT"] = le.fit_transform(df_st_verify_data["PendienteST"])
df_st_verify_data.head()

In [ ]:
# eliminar las columnas originales del dataframe
df_st_verify_data.drop('TipoDolorPecho', axis = 1, inplace=True)
df_st_verify_data.drop('ECGRep', axis = 1, inplace=True)
df_st_verify_data.drop('PendienteST', axis = 1, inplace=True)
df_st_verify_data.head()

In [ ]:
# verificamos dataset
print(' - Cantidad de Filas y columnas:', df_st_verify_data.shape, '\n')

In [ ]:
# identificar valores faltantes
df_st_verify_data.isnull().values.any()

In [ ]:
print(df.isnull().sum())

In [ ]:
# realizamos las predicciones sobre el conjunto objetivo, se elije el modelo "algoritmo SVM"
conjunto_objetivo = df_st_verify_data
predicciones = algoritmo.predict(conjunto_objetivo)

In [ ]:
# se descarga el archivo con las predicciones
nombre_csv = "pcp_respuestas_NoeliaMackinnon.csv"

# se agrega columna 'EnfermedadCorazon'
df_st_verify_data['EnfermedadCorazon']= predicciones

# pasar a csv
dfnm = df_st_verify_data.to_csv(nombre_csv, index=False)

# descargar archivo
from google.colab import files
files.download(nombre_csv)

<div class="markdown-google-sans">

## Realizado por
</div>

<a href="https://uy.linkedin.com/in/noelia-mackinnon" target="_blank">Noelia Mackinnon</a>
